In [1]:
import pydicom
import fnmatch
import os
import numpy as np
from pydicom.data import get_testdata_files
from pydicom.filereader import read_dicomdir
from scipy.spatial.distance import cdist
from os.path import dirname, join
from ipywidgets.widgets import * 
import ipywidgets as widgets
import cv2
import collections
from pprint import pprint
import matplotlib.pyplot as plt
import sys
import glob
import csv
import pandas as pd
from myfunctions import get_generated_mask,read_record,find_overlap_area,apply_mask,create_overlay

In [2]:
MASK_COLOR = (0.0,0.0,1.0)
accepted_overlap=2000

In [7]:
# Enter folder data 
# folder_data=path='/dbfs/mnt/data-mahdi'
folder_data='/Users/Engin/Documents/Edwards/Projects/Echo/MS_test'
foldPath_split=folder_data.split('/')
result_fold='/'.join(foldPath_split[:-1])
mask_path=os.path.join(os.getcwd(),foldPath_split[-1]+'_'+'generated_mask')
overlay_alpha_blended_path=os.path.join(result_fold,foldPath_split[-1]+'_'+'alpha_blend_folder')
overlay_video_path=os.path.join(result_fold,foldPath_split[-1]+'_'+'avi_folder')
unsuccess_records_path=os.path.join(result_fold,'unsuccess_records')
unsuccess=[]

unsuccess=[]
if not os.path.exists(unsuccess_records_path):
    os.mkdir(unsuccess_records_path)
if not os.path.exists(mask_path):
    os.mkdir(mask_path)
if not os.path.exists(overlay_video_path):
    os.mkdir(overlay_video_path) 
if not os.path.exists(overlay_alpha_blended_path):
    os.mkdir(overlay_alpha_blended_path)
path_testrecords=[]
unsuccess_records_file=os.path.join(unsuccess_records_path,'unsuccess_records.csv')
for path, subdirs, files in os.walk(folder_data):
    for name in files:
#         if name.startswith('IM'):  
          if fnmatch.fnmatch(name, '*IM*'):
            path_testrecords.append([os.path.join(path, name)])

#   run for all 
#  start with current generated mask. Pass records that are already generated.
# recorded_models=get_mask_path(mask_path)
available_model,available_folder,available_masks=get_generated_mask(mask_path)
# ,available_folder=get_generated_mask(mask_path)
# print(available_folder)

for path in sorted(path_testrecords):     
    path=path[0]
    print(path)
    folder_name=path.split('/')[-3]
    file_name=path.split('/')[-1]  
    print(file_name,folder_data)
        
    avi_fname=folder_name+'_'+file_name+'.avi'

    avi_file_path=os.path.join(overlay_video_path,avi_fname)
    avi_blend_path=os.path.join(overlay_alpha_blended_path,avi_fname)

#     if folder_name not in available_folder:
    data,feats_rec,val=read_record(path,file_name,folder_name)
    if val:
        is_color=data.shape[-1]==3
        modelName='_'.join([feats_rec[1],feats_rec[0]])
              
        
        if modelName in available_masks:
            
            rel_masks=available_masks[modelName]
            index1=0
            sample_img=data[0,:,:,:] if is_color else data[0,:,:]
            grayimg=cv2.cvtColor(sample_img, cv2.COLOR_BGR2GRAY) if is_color else sample_img
#             print(sample_img.shape)
#             Mask_n=
            indexx=0
#             for i,Mask_n in enumerate(rel_masks):
            
#                 s=evaluate_mask(Mask_n,grayimg,100)
#                 if s < max_score:
#                     max_score=s
#                     indexx=i
#             print(max_score,indexx)
            max_overlap_score,indexx=find_overlap_area(grayimg,rel_masks)
            print(max_overlap_score)

            if max_overlap_score < accepted_overlap:
                print('Mask is available for this manufacturer and model but is not a good fit for {}'.format(folder_name+'_'+file_name))
                unsuccess.append([path,max_overlap_score])               
            mask_r=rel_masks[indexx]
            frame_width,frame_height=data.shape[2],data.shape[1]
            out = cv2.VideoWriter(avi_file_path, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (frame_width,frame_height))
            out_blend = cv2.VideoWriter(avi_blend_path, cv2.VideoWriter_fourcc('M', 'J', 'P', 'G'), 10, (frame_width,frame_height))
            print('Creating avi file')

            for j in range(data.shape[0]):       
                img_=data[j,:,:,:] if is_color else data[j,:,:]


                res_masked_img=apply_mask(img_,mask_r,is_color)
                blended_img=create_overlay(img_,mask_r,is_color)

#                 mask_stack = np.dstack([mask]*3)  # Create 3-channel alpha mask
#                 img_clr = img_ if cm else np.dstack([img_]*3)
#                 mask_stack  = mask_stack.astype('float32') / 255.0          # Use float matrices, 
#                 norm_img    = img_clr.astype('float32') / 255.0
#                 res_masked_img = (mask_stack * norm_img)

#                 blend_img = (mask_stack * norm_img) + ((1-mask_stack) * MASK_COLOR) # Blend
#                 masked_img = (masked_img * 255).astype('uint8') 

                # vis2 = cv2.cvtColor(ra_depth.astype(np.uint8), cv2.COLOR_GRAY2BGR)
                vis1 = cv2.resize(res_masked_img, dsize=(frame_width,frame_height))
                out.write(vis1)

                vis2 = cv2.resize(blended_img, dsize=(frame_width,frame_height))
                out_blend.write(vis2)
        else:
            print('Mask is not available for this manufacturer and model')
                        
out.release()
out_blend.release()
# -------------------------------------------------
# import csv
# with open(unsuccess_records_file) as result_file:
#     wr = csv.writer(result_file, dialect='excel')
#     wr.writerows(unsuccess)


/Users/Engin/Documents/Edwards/Projects/Echo/deidentifier_vs/MS_test_generated_mask/Acuson_Cypress_MS_test_87690_IM28.png
/Users/Engin/Documents/Edwards/Projects/Echo/deidentifier_vs/MS_test_generated_mask/GE Healthcare Ultrasound_Vivid iq_87568_IM13.png
/Users/Engin/Documents/Edwards/Projects/Echo/deidentifier_vs/MS_test_generated_mask/GE Healthcare Ultrasound_Vivid iq_87568_IM0.png
/Users/Engin/Documents/Edwards/Projects/Echo/deidentifier_vs/MS_test_generated_mask/GE Healthcare Ultrasound_Vivid iq_87568_IM1.png
/Users/Engin/Documents/Edwards/Projects/Echo/deidentifier_vs/MS_test_generated_mask/GE Healthcare Ultrasound_Vivid iq_87568_IM2.png
/Users/Engin/Documents/Edwards/Projects/Echo/deidentifier_vs/MS_test_generated_mask/GE Healthcare_Vivid i_MS_test_87626_IM20.png
/Users/Engin/Documents/Edwards/Projects/Echo/MS_test/bin2/87626_IM20
87626_IM20 /Users/Engin/Documents/Edwards/Projects/Echo/MS_test
The image has 422 x 636 voxels
8973
Creating avi file
/Users/Engin/Documents/Edwards/Pr

In [4]:
available_model

[]

In [5]:
available_model,available_folder,available_masks=get_generated_mask(mask_path)

In [6]:
mask_path

'/Users/Engin/Documents/Edwards/Projects/Echo/deidentifier_vs/MS_test_generated_mask'